In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [2]:
'''

get file in jupyter cell
%load filename.py

save to file from jupyter cell (will overwrite)
%%writefile filename.py

'''

'\nget file in jupyter cell\n%load filename.py\n\nsave to file from jupyter cell (will overwrite)\n%%writefile filename.py\n'

In [2]:
import glob

In [45]:
#%%writefile build_financial_df.py
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
import glob

#read file to access headers for the group of CSVs you want to use in DF
financial_headers = pd.read_csv('/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/fin_head.csv', sep=",", header=None)
financial_headers_list = list(financial_headers.iloc[:,0])

#read all 20 txt files (from 2 directories) & concatenate to one df
path1 = r'/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/all_80to18/80to98'
path2 = r'/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/all_80to18/00to18'
files1 = glob.glob(path1 + "/*.txt")
files2 = glob.glob(path2 + "/*.txt")


li = []

for filename in files1:
    df = pd.read_csv(filename, index_col=None, header=None, sep='|', names=financial_headers_list, dtype={'DISTRICT':str})
    x = filename.replace('/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/all_80to18/80to98/weball', '').replace(".txt",'')
    df['YEAR'] = int('19'+ x)
    li.append(df)
    
for filename in files2:
    df = pd.read_csv(filename, index_col=None, header=None, sep='|', names=financial_headers_list, dtype={'DISTRICT':str})
    x = filename.replace('/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/all_80to18/00to18/weball', '').replace(".txt",'')
    df['YEAR'] = int('20'+ x)
    li.append(df)
    
financial_df_raw = pd.concat(li, axis=0, ignore_index=0)
financial_df_raw['LAST_NAME'] = financial_df_raw['NAME'].str.replace(',.*', '')
financial_df_raw["ELEC_ID"] = financial_df_raw['YEAR'].astype(str) + "-"+ financial_df_raw['STATE'].astype(str) +"-"+ financial_df_raw['DISTRICT'].astype(str)

'''
attempt to merge with candidate data files... changed mind for now
path = r'/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/financial/cn'
files = glob.glob(path + "/*.txt")
cand_cols = ['ID', "x1", "x2", 'ELEC_YEAR',"x3",'OFFICE', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15']
li = []

for filename in files:
    df = pd.read_csv(filename, index_col=None, header=None, sep='|', names=cand_cols)
    li.append(df)
    
#candidate_df_raw = pd.concat(li, axis=0, ignore_index=0)
#candidate_df = candidate_df_raw[['ID','ELEC_YEAR','OFFICE']]
#join1 = financial_df_raw.merge(candidate_df, how='inner', left_on='FEC_ID', right_on='ID')
#financial_df_plus = x123.drop('ID', axis=1)
'''

financial_df_raw["ELEC_ID"] = financial_df_raw['YEAR'].astype(str) + "-"+ financial_df_raw['STATE'].astype(str) +"-"+ financial_df_raw['DISTRICT'].astype(str)
#filter out presidential candidates
fin_cut_1 = financial_df_raw[financial_df_raw['STATE']!='00']
fin_cut_1['ELEC-CAND'] = fin_cut_1["ELEC_ID"].astype(str) +"-"+ fin_cut_1["LAST_NAME"].astype(str)
#filter out repeats in ELEC-CAND... may want to add some back because of last name issue
something = fin_cut_1['ELEC-CAND'].value_counts()
something
new_df = pd.DataFrame()
new_df['filtered'] = something[something > 1]
finance_df = fin_cut_1[~fin_cut_1['ELEC-CAND'].isin(new_df.reset_index()['index'])]
finance_df

'''
for later consideration :
mystery_df = fin_cut_1[fin_cut_1['ELEC-CAND'].isin(new_df.reset_index()['index'])]

'''


Overwriting build_financial_df.py


In [170]:
financial_df_raw.columns

Index(['FEC_ID', 'NAME', 'INC_STATUS', 'PTY_CD', 'PARTY', 'TTL_RECEIPTS',
       'TRANS_FROM_AUTH', 'TTL_DISB', 'TRANS_TO_AUTH', 'CASH_BEGIN',
       'CASH_END', 'CAND_CONTRIB', 'CAND_LOANS', 'OTHER_LOANS',
       'CAND_LOAN_REPAY', 'OTHER_LOAN_REPAY', 'DEBTS_OWED_BY',
       'TTL_INDIV_CONTRIB', 'STATE', 'DISTRICT', 'SPEC_ELEC', 'PRIM_ELEC',
       'RUN_ELEC', 'GEN_ELEC', 'GEN_ELEC_PRECENT', 'OTHER_POL_CMTE_CONTRIB',
       'POL_PTY_CONTRIB', 'CVG_END_DT', 'INDIV_REFUNDS', 'CMTE_REFUNDS',
       'YEAR', 'LAST_NAME'],
      dtype='object')

In [32]:
mystery_df = fin_cut_1[fin_cut_1['ELEC-CAND'].isin(new_df.reset_index()['index'])]
mystery_df[['FEC_ID','Name','Party',
'ELEC-CAND']]

,FEC_ID,NAME,INC_STATUS,PTY_CD,PARTY,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,CASH_BEGIN,...,GEN_ELEC_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,YEAR,LAST_NAME,ELEC_ID,ELEC-CAND
245,H6CA37115,"PETTIS, RONALD E.",NaN,1,DEM,0.00,0.00,0.00,0.00,0.00,...,NaN,0.00,0.00,03/27/1979,0.00,0.00,1980,PETTIS,1980-CA-37,1980-CA-37-PETTIS
247,H6CA37123,"PETTIS, SHIRLEY N.",C,2,REP,0.00,0.00,3638.00,0.00,0.00,...,NaN,0.00,0.00,07/31/1979,0.00,0.00,1980,PETTIS,1980-CA-37,1980-CA-37-PETTIS
266,H0CA41029,"WILSON, BOB",O,1,DEM,364594.00,0.00,362792.00,0.00,0.00,...,43.0,0.00,0.00,12/31/1980,0.00,0.00,1980,WILSON,1980-CA-41,1980-CA-41-WILSON
273,H6CA41059,"WILSON, BOB CONGRESSMAN",I,2,REP,1620.00,0.00,33154.00,0.00,0.00,...,NaN,0.00,0.00,12/31/1980,0.00,0.00,1980,WILSON,1980-CA-41,1980-CA-41-WILSON
563,H0IL16036,"ANDERSON, STEVEN CRAIG",O,2,REP,30933.00,0.00,30759.00,0.00,0.00,...,0.0,0.00,0.00,12/31/1980,0.00,0.00,1980,ANDERSON,1980-IL-16,1980-IL-16-ANDERSON
566,H6IL16025,"ANDERSON, JOHN B",I,2,REP,6028.00,0.00,25772.00,0.00,0.00,...,NaN,0.00,0.00,06/30/1979,0.00,0.00,1980,ANDERSON,1980-IL-16,1980-IL-16-ANDERSON
629,H6IN09036,"HAMILTON, LEE HERBERT",I,1,DEM,113260.00,0.00,122674.00,0.00,0.00,...,64.0,0.00,0.00,12/31/1980,0.00,0.00,1980,HAMILTON,1980-IN-09,1980-IN-09-HAMILTON
630,H8IN09024,"HAMILTON, FRANK ISGRIGG JR.",NaN,2,REP,0.00,0.00,0.00,0.00,0.00,...,NaN,0.00,0.00,12/31/1980,0.00,0.00,1980,HAMILTON,1980-IN-09,1980-IN-09-HAMILTON
653,H0KS02030,"KEYS, SAMUEL R",NaN,1,DEM,95868.00,0.00,89798.00,0.00,0.00,...,46.0,0.00,0.00,12/31/1980,0.00,0.00,1980,KEYS,1980-KS-02,1980-KS-02-KEYS
654,H6KS02029,"KEYS, MARTHA",C,1,DEM,209.00,0.00,6815.00,0.00,0.00,...,NaN,0.00,0.00,12/31/1980,0.00,0.00,1980,KEYS,1980-KS-02,1980-KS-02-KEYS


In [58]:
#finance_df.loc[finance_df['NAME'] == 'AYOTTE, KELLY']
x = finance_df['NAME'].str.contains('AYOTTE*', na=False)
finance_df[x]
#df.loc[df['column_name'] == some_value]

,FEC_ID,NAME,INC_STATUS,PTY_CD,PARTY,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,CASH_BEGIN,...,GEN_ELEC_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS,YEAR,LAST_NAME,ELEC_ID,ELEC-CAND
3480,S0NH00235,"AYOTTE, KELLY A",I,2,REP,62422.00,0.0,101203.39,0.0,176170.0,...,NaN,0.0,0.0,12/31/2018,216.0,0.0,2018,AYOTTE,2018-NH-00,2018-NH-00-AYOTTE
2765,S0NH00235,"AYOTTE, KELLY A",I,2,REP,1767421.00,644.0,809702.00,0.0,691987.0,...,NaN,646338.0,0.0,12/31/2014,8755.0,0.0,2014,AYOTTE,2014-NH-00,2014-NH-00-AYOTTE
2737,S0NH00235,"AYOTTE, KELLY A",I,2,REP,13980279.89,955657.0,14954757.26,0.0,2039165.0,...,NaN,2893531.1,50800.0,12/31/2016,150809.0,35890.0,2016,AYOTTE,2016-NH-00,2016-NH-00-AYOTTE
3069,S0NH00235,"AYOTTE, KELLY A",I,2,REP,599631.00,11510.0,328716.00,0.0,348770.0,...,NaN,290861.0,0.0,12/31/2012,500.0,0.0,2012,AYOTTE,2012-NH-00,2012-NH-00-AYOTTE
2965,S0NH00235,"AYOTTE, KELLY A",O,2,REP,5389780.00,507307.0,5041009.00,0.0,0.0,...,60.0,1388591.0,42600.0,12/31/2010,13490.0,1000.0,2010,AYOTTE,2010-NH-00,2010-NH-00-AYOTTE


In [31]:
y = fin_cut_1["ELEC_ID"].astype(str) +"-"+ fin_cut_1["LAST_NAME"].astype(str)
y

0         1980-AK-00-WHITTAKER
1           1980-AK-00-PARNELL
2           1980-AK-00-HENSLEY
3            1980-AK-00-HOPSON
4             1980-AK-00-RODEY
5             1980-AK-00-YOUNG
6            1980-AK-00-COWPER
7              1980-AL-01-PAUL
8            1980-AL-01-NOONAN
9           1980-AL-01-EDWARDS
10            1980-AL-02-WYATT
11         1980-AL-02-MITCHELL
12        1980-AL-02-DICKINSON
13          1980-AL-03-NICHOLS
14           1980-AL-04-BEVILL
15           1980-AL-05-FLIPPO
16            1980-AL-05-POTTS
17         1980-AL-06-CLIFFORD
18          1980-AL-06-JOHNSON
19         1980-AL-06-BUCHANAN
20            1980-AL-06-SMITH
21           1980-AL-07-SHELBY
22          1980-AL-07-SCRUGGS
23             1980-AL-07-GRAY
24        1980-AR-01-ALEXANDER
25             1980-AR-02-REID
26          1980-AR-02-WHITTEN
27            1980-AR-02-ALLEN
28          1980-AR-02-BRANDON
29             1980-AR-02-RUSS
                 ...          
3143         2010-WA-00-BENTON
3144    

In [32]:
type(y)

pandas.core.series.Series

In [33]:
z = y.value_counts()
z[z>1]

2010-LA-02-CARTER       3
1992-VA-07-SLAUGHTER    3
2018-TX-29-GARCIA       3
2018-MI-13-CONYERS      3
2008-LA-02-CARTER       3
2010-AK-00-MURKOWSKI    2
1990-NC-01-JONES        2
1996-MO-08-EMERSON      2
2004-NY-11-CLARKE       2
2002-NM-02-SMITH        2
1996-FL-03-BROWN        2
1992-DE-00-WOO          2
2016-VA-00-MARSHALL     2
2018-AZ-02-HEINZ        2
1980-KY-05-ROGERS       2
2008-MO-01-CLAY         2
1996-VA-00-WARNER       2
1990-WY-00-VINICH       2
2012-MT-00-DAINES       2
2000-MO-06-DANNER       2
2012-TX-35-ALVARADO     2
1988-MN-07-HANSON       2
2014-MT-00-STAPLETON    2
1996-NY-12-VELAZQUEZ    2
1990-MS-05-SMITH        2
1996-IN-00-BAYH         2
1980-CA-41-WILSON       2
1980-IL-16-ANDERSON     2
2018-FL-03-WELLS        2
2014-RI-01-LYNCH        2
                       ..
2010-VA-00-WARNER       2
1992-ND-00-DORGAN       2
1984-NC-09-MARTIN       2
2016-WA-07-MCDERMOTT    2
1980-PA-14-MOORHEAD     2
2008-FL-17-MEEK         2
2008-IN-07-CARSON       2
1994-FL-03-B